In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

%matplotlib inline

In [29]:
act = pd.read_csv('data/atusact_2014.dat')
cps = pd.read_csv('data/atuscps_2014.dat')
resp = pd.read_csv('data/atusresp_2014.dat')
rost = pd.read_csv('data/atusrost_2014.dat')
atussum = pd.read_csv('data/atussum_2014.dat') 
who = pd.read_csv('data/atuswho_2014.dat')

In [30]:
resp.head()
#resp['TRTHHFAMILY'] 
#resp['TRTNOCHILD']
new_data = pd.concat([resp['TRTHHFAMILY'], resp['TRTNOCHILD']], axis=1, keys=['1', '2'])
new_data.corr()

,1,2
1,1.000000,0.168112
2,0.168112,1.000000


In [31]:
rost.head()

,TUCASEID,TULINENO,TEAGE,TERRP,TESEX,TXAGE,TXRRP,TXSEX
0,20140101140007,1,20,19,2,0,0,0
1,20140101140007,2,19,21,1,0,0,0
2,20140101140007,3,0,22,1,0,0,0
3,20140101140011,1,16,18,2,0,0,0
4,20140101140011,2,33,24,2,0,0,0


In [49]:
atussum.rename(columns={'TEAGE':'Age', 'TESEX':'Sex', 'tucaseid':'ID', 't120302':'drug_use','t120301':'relaxing_thinking', 't050101':'working'}, inplace=True)
atussum.rename(columns={'t120303':'TV','t120305':'Radio','t140101':'religious','t160101':'phone', 't120313':'writing'}, inplace=True)
atussum.rename(columns={'t120404':'gambling', 't010101':'sleeping','t500106':'gap', 't180501':'work_travel'}, inplace=True)
interesting_data = pd.concat([atussum['ID'], atussum['Age'], atussum['Sex'], atussum['drug_use'], atussum['relaxing_thinking']], axis=1, keys=['ID', 'Age', 'Sex', 'drug_use', 'relaxing_thinking'])
interesting_data['working'] = atussum['working']
interesting_data['TV'] = atussum['TV']
interesting_data['Radio'] = atussum['Radio']
interesting_data['religious'] = atussum['religious']
interesting_data['phone'] = atussum['phone']
interesting_data['writing'] = atussum['writing']
interesting_data['gambling'] = atussum['gambling']
interesting_data['sleeping'] = atussum['sleeping']
interesting_data['gap'] = atussum['gap']
interesting_data['work_travel'] = atussum['work_travel']
atussum = atussum.corr()
for x in atussum['sleeping']:
    if x > .40 or x < -.40:
        print(x)

-0.400442745477
-0.402282784184
1.0
-0.532681815669
-0.483902504237


In [39]:
interesting_data.corr()

,ID,Age,Sex,drug_use,relaxing_thinking,working,TV,Radio,religious,phone,writing,gambling,sleeping,gap
ID,1.000000,-0.007162,-0.017115,0.010170,-0.005620,-0.007540,-0.004635,-0.012316,0.012055,-0.015804,0.001599,-0.010551,0.010324,-0.017501
Age,-0.007162,1.000000,0.051919,-0.012528,0.108553,-0.172342,0.252330,0.054733,0.061291,0.075352,0.009504,0.033953,-0.041956,0.099584
Sex,-0.017115,0.051919,1.000000,0.002179,-0.019879,-0.128552,-0.084646,-0.019857,0.051106,0.094356,-0.016057,-0.013673,0.046865,0.018821
drug_use,0.010170,-0.012528,0.002179,1.000000,-0.003088,0.007088,-0.004657,-0.001341,-0.015033,0.015161,-0.002077,-0.002712,-0.025801,-0.009053
relaxing_thinking,-0.005620,0.108553,-0.019879,-0.003088,1.000000,-0.079454,-0.047878,0.021202,0.010611,-0.012494,-0.007679,-0.008832,-0.004910,-0.000952
working,-0.007540,-0.172342,-0.128552,0.007088,-0.079454,1.000000,-0.313846,-0.023008,-0.132238,-0.056621,-0.006519,-0.023974,-0.332465,-0.067800
TV,-0.004635,0.252330,-0.084646,-0.004657,-0.047878,-0.313846,1.000000,-0.006814,-0.006948,-0.007279,-0.013734,-0.003788,0.035563,-0.019328
Radio,-0.012316,0.054733,-0.019857,-0.001341,0.021202,-0.023008,-0.006814,1.000000,0.025761,-0.005729,-0.000873,-0.003858,-0.025609,0.003671
religious,0.012055,0.061291,0.051106,-0.015033,0.010611,-0.132238,-0.006948,0.025761,1.000000,0.017264,-0.005689,0.000671,0.040478,0.005753
phone,-0.015804,0.075352,0.094356,0.015161,-0.012494,-0.056621,-0.007279,-0.005729,0.017264,1.000000,-0.005019,-0.005444,-0.043124,0.005804
